In [1]:
import config
import pandas as pd
import numpy as np
import itertools
import googlemaps
from datetime import 
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, LinearColorMapper
from bokeh.palettes import Viridis10
from bokeh.models import LogColorMapper, LogTicker, ColorBar

In [ ]:
# convert wsg84 to web mercator
def wgs84_to_web_mercator(df, lon="LON", lat="LAT"):
      k = 6378137
      df["x"] = df[lon] * (k * np.pi/180.0)
      df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
      return df

In [ ]:
# define lat lon steps for grid
lat_diff = 0.1
lon_diff = 0.2
#  define grid
lats = np.arange(53.00, 54.37, lat_diff)
lons = np.arange(8.63, 11.18, lon_diff)
# get grid
combinations = list(itertools.product(lats, lons))
# grid to df
df = pd.DataFrame(combinations, columns =['LAT', 'LON']) 
print(df.shape)
# define origin
origin = [[53.558797], [9.962744]]
origin = {'LAT': [53.558797], 'LON': [9.962744]}
df_origin = pd.DataFrame(data=origin)
# get tuples for google directions matrix
tuples = [tuple(x) for x in df.values]
# convert coordinates
df2 = wgs84_to_web_mercator(df)
df_origin2 = wgs84_to_web_mercator(df_origin)

In [202]:
# define maps api key
gmaps = googlemaps.Client(key='YOUR_API_KEY')

# Geocoding an address
geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

# Look up an address with reverse geocoding
reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# Request directions via public transit
directions_result = gmaps.distance_matrix(tuple(df_origin.values[0]),
                                     tuples,
                                     mode="driving")

# Extract duration for each item

duration = []
for element in directions_result['rows'][0]['elements']:
    if element['status']== "OK":
        duration.append(element['duration']['value']/60)
    else:
        duration.append(np.nan)

ApiError: MAX_ELEMENTS_EXCEEDED

In [195]:
# PLOT

Viridis10.reverse()

output_file("tile.html")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)

# range bounds supplied in web mrcator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

# add duration to df
df2['duration'] = duration
source = ColumnDataSource(df2)
source_origin = ColumnDataSource(df_origin2)

color_mapper = LinearColorMapper(palette=Viridis10, low=df2['duration'].min(), high=df2['duration'].max())

p.circle(x="x", y="y", size=15,
         fill_color = {'field'  : 'duration',  'transform': color_mapper},
         line_color=None,
         source=source)
p.circle(x="x", y="y", size=15, fill_color="red", line_color=None, fill_alpha=0.8, source=source_origin)

color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

p.add_layout(color_bar, 'right')

show(p)

In [ ]:
#  example image for testing

![title](bokeh_plot.png)